In [ ]:
import sys
!{sys.executable} -m pip install transformers
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install torchvision

In [1]:

from transformers import AutoTokenizer, AutoModel
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from torchvision import datasets
import torch
import torch.nn as nn
import torch.optim as optim
import csv


def preparedata(sentences, tokennumbers):
    
    RetVal=[]
    
    inputs=tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    outputs=aModel(**inputs)
    
    for i, tokens in enumerate(outputs[0]):
        RetVal.append(tokens[int(tokennumbers[i])])
    
    return RetVal

def casetonumber(case):
    switch = {
        "Nom": 0,
        "Acc": 1,
        "Ins": 2,
        "Ine": 3, 
        "Sup": 4,
        "Sub": 5
    }
    
    return switch.get(case, "Invalid case")



In [2]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
aModel = AutoModel.from_pretrained("bert-base-multilingual-cased")

file1 = open("outputtrainrevised.csv")
train_tsv=csv.reader( file1, delimiter="\t")

file2 = open("outputtestrevised.csv")
test_tsv=csv.reader( file2, delimiter="\t")

training_data=[]
test_data=[]


for i ,obj in enumerate(train_tsv, 0):
    training_data.append([])
    training_data[i].append(obj[0])
    training_data[i].append(obj[3])
    training_data[i].append(casetonumber(obj[4]))

    
for i, obj in enumerate(test_tsv):
    test_data.append([])
    test_data[i].append(obj[0])
    test_data[i].append(obj[3])
    test_data[i].append(casetonumber(obj[4]))


trainloader=torch.utils.data.DataLoader(training_data, batch_size=16, shuffle=True)
testloader=torch.utils.data.DataLoader(test_data, batch_size=16, shuffle=True)
file1.close()
file2.close()


In [ ]:
#test=tokenizer("Ez egy testmondat.", return_tensors="pt")
test=tokenizer(["Ez egy testmondat.", "Ez is az.", "Emez is az."],padding=True, truncation=True, return_tensors="pt")
out=aModel(**test)
print(out[0][1][0])

In [3]:

class SimpleClassifier(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim):
        super().__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, X):
        h = self.input_layer(X)
        h = self.relu(h)
        out = self.output_layer(h)
        return out

In [4]:
Net = SimpleClassifier(
    input_dim=768,
    output_dim=6,
    hidden_dim=50
)
Net
Net=Net.cuda()

In [5]:


def createLoss():
    return nn.CrossEntropyLoss()
def createOptimizer():
    return torch.optim.SGD(Net.parameters(), lr=1e-1, momentum=0.9, nesterov=True, weight_decay=1e-4)
numEpoch = 10
def creatScheduler():
    return torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, numEpoch, eta_min=1e-2)

In [13]:
def train(epoch):
    Net.train()
    running_loss=0.0
    correct = 0.0
    total=0
    for i, data in enumerate(trainloader, 0):
        sentences, numbers, labels = data
        inputs=torch.stack(preparedata(sentences,numbers))
        inputs=inputs.cuda()
        labels=labels.cuda()
        optimizer.zero_grad()
        outputs=Net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            running_loss +=loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)
            
            
    tr_loss = running_loss/i
    tr_corr = correct/total*100
    print("Train epoch "+str(epoch+1)+"  correct: "+ str(tr_corr))
    return tr_loss, tr_corr

def val(epoch):
    Net.eval()
    running_loss=0.0
    correct = 0.0
    total=0
    for i, data in enumerate(testloader, 0):
        sentences, numbers, labels = data
        inputs=torch.stack(preparedata(sentences,numbers))
        labels=labels.cuda()
        inputs=inputs.cuda()
        with torch.no_grad():
            outputs=Net(inputs)
            loss = criterion(outputs, labels)
            running_loss +=loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)
            
            
    val_loss = running_loss/i
    val_corr = correct/total*100
    print("Test epoch "+str(epoch + 1)+"loss: "+ str(running_loss / i)+ "correct: "+  str(val_corr))
    return val_loss, val_corr

In [ ]:

train_accs = []
train_losses = []
val_accs = []
val_losses = []
best_acc = 0
torch.manual_seed(32)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark=False
criterion = createLoss()
optimizer = createOptimizer()
scheduler = creatScheduler()
for epoch in range(numEpoch):
    #Train
    loss, acc = train(epoch)
    train_accs.append(acc)
    train_losses.append(loss)
    #val
    loss, acc = val(epoch)
    val_accs.append(acc)
    val_losses.append(loss)
    scheduler.step()
    print(acc)
    if acc>best_acc:
        best_acc = acc
        print("Best model so far")
        torch.save(Net, "model.pth")